In [3]:
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

queen_bed = {
    "name": "queen_bed",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    "name": "bedroom_1",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_1, outside]

all_doors = [door_a, door_b]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "outside": [door_b], ## update relation every time a new test room is added
    "door a": [game_room, bedroom_1],
    "bedroom_1": [queen_bed, door_b],
    "queen_bed": [key_b],
    "door b": [bedroom_1 ,outside]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [6]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"]) ##### first step of the room

    ##### Starts in game_room (initialized in state)
    ##### value gets update when go_to_next_room functions gets called

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room ##### stores in a variable the room for better readibility 
    if(game_state["current_room"] == game_state["target_room"]): 
        print("Congrats! You escaped the room!") ##### validates if you win or not
    else:
        print("You are now in " + room["name"]) ##### uses the name value of the game_room dict, this would be something like  game_state["current_room"]["name"]
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip() 
        if intended_action == "explore":
            explore_room(room) ##### shows the items in the room depending on the current room
            play_room(room) ##### restart the process with print "You are now in room" 
        elif intended_action == "examine": 
            examine_item(input("What would you like to examine?").strip()) ##### type a value of the ones specified in explore function
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]] ##### defines a list of the items in current room (need to update object relations for each room)
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]] ##### find relations of items to get the specific room
    for room in connected_rooms: 
        if(not current_room == room): ##### loops through the rooms to check if the current room is not the one that object relations points to and returned it 
            return room

def examine_item(item_name): ##### gets an input to examine
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"] ##### defined as variable for future use in get_door_next_room
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]: ##### loop items list of current room values - for game_room would be couch, piano and door a
        if(item["name"] == item_name): ##### validates if input matches the object relations values 
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"): 
                have_key = False
                for key in game_state["keys_collected"]: ##### checks for keys of the dict values
                    if(key["target"] == item): ##### validates if that specific key opens that door (item)
                        have_key = True ##### turns the state to true if the key opens that door (item)
                if(have_key):
                    output += "You unlock it with a key you have." ##### appends the string value 
                    next_room = get_next_room_of_door(item, current_room) ##### item is a door - apply get_next_room_of_door function
                else:
                    output += "It is locked but you don't have the key." ##### have_key remains false and blocks the process
            else: ##### this happens when the item is not a door
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):  ### when the item is at least one time present in object relations dict
                    item_found = object_relations[item["name"]].pop() ##### removes the key from the objet relations of the specific item, if all the keys are collected all the related items should be without value
                    game_state["keys_collected"].append(item_found) ##### adds the key to the collected state
                    output += "You find " + item_found["name"] + "." ##### shows the key name
                else:
                    output += "There isn't anything interesting about it." ##### item is selected and key is not present in there
            print(output)
            break 

    if(output is None):
        print("The item you requested is not found in the current room.") ##### write an item that is not the one in the explore function
    
    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes'): ##### if next_room exists (defined after key is used when get_next_room fucntion is applied)
        play_room(next_room) ##### uses the target value of the item when the key is used 
    else:
        play_room(current_room) ##### leaves you in the previous room if you say no

In [7]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore' or 'examine'?1
Not sure what you mean. Type 'explore' or 'examine'.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type 'explore' or 'examine'?door a
Not sure what you mean. Type 'explore' or 'examine'.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type 'explore' or 'examine'?couch
Not sure what you mean. Type 'explore' or 'examine'.
You a